# Tokenization

### BertTweet

- fastBPE
- 64K subword

### Twilbert
- SentencePiece (fastBPE)
- 30k subword 

In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob

num_files = 100
tweet_files = glob("../../data/filtered_tweets/*.txt")

train_files = tweet_files[:2]


tweets = list([x.strip("\n") for x in open(tweet_files[0])])[:100_000]

In [2]:
len(tweets)

100000

In [3]:
from tokenizers import SentencePieceUnigramTokenizer, SentencePieceBPETokenizer, BertWordPieceTokenizer, ByteLevelBPETokenizer

tokenizer = SentencePieceBPETokenizer()#replacement="_")

In [4]:
from finetune_vs_scratch.preprocessing import special_tokens
from finetune_vs_scratch.tokenizer import tokenizer_special_tokens

#tokenizer.add_special_tokens(tokenizer_special_tokens)
tokenizer.train_from_iterator(
    tweets,
    vocab_size=30_000,
    min_frequency=5,
    show_progress=True,
    limit_alphabet=500,
    special_tokens=tokenizer_special_tokens,
)

In [5]:
tokenizer_path = "./sentence-piece-tokenizer"
!mkdir $tokenizer_path
vocab_file, merges_file = tokenizer.save_model(tokenizer_path)

In [9]:
from transformers import PreTrainedTokenizerFast

transformer_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer
)

In [11]:

transformer_tokenizer("@usuario tugo bierno skere comunista")

{'input_ids': [534, 764, 624, 569, 1206, 13253, 926, 13757], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
inv_vocab = {v:k for k, v in transformer_tokenizer.vocab.items()}
tok_ids = transformer_tokenizer("@usuario tugo bierno skere comunista")["input_ids"]

for tok in tok_ids:
    print(tok, " ---> ", inv_vocab[tok])

534  --->  ▁@usuario
764  --->  ▁tu
624  --->  go
569  --->  ▁b
1206  --->  ierno
13253  --->  ▁sk
926  --->  ere
13757  --->  ▁comunista


In [10]:
%%timeit

transformer_tokenizer(tweets[:1000]);

None

23.3 ms ± 572 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
%%timeit

t_tokenizer(tweets[:1000]);

None

501 ms ± 7.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit

roberta_tokenizer(tweets[:1000]);

None

24.9 ms ± 1.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
%%timeit

tokenizer.encode_batch(tweets[:1000])

None

18.1 ms ± 613 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
%%timeit

bertweet_tokenizer(tweets[:1000])

None

Token indices sequence length is longer than the specified maximum sequence length for this model (129 > 128). Running this sequence through the model will result in indexing errors


103 ms ± 10.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


La implementación nuestra es muy muy mala